## DEPENDANT ON TWITTERFINGERS AND NEWSSTORIES

In [1]:
import datetime
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

In [2]:
def getTargets():
    targets = pd.read_csv('targetSp.csv')
    return targets.Symbol.to_numpy()

In [3]:
def getTweets():
    return pd.read_csv('data/' +(datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d') + '/tweets.csv')

In [4]:
def getHeadlines():
    return pd.read_csv('data/' +(datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d') + '/headlines.csv')

In [5]:
def getPipe():
    tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
    model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
    pipe = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
    return pipe

In [6]:
def getSentiments(data, pipe, column, typ):
    sent = pd.DataFrame(columns=['target','label','score'])
    for row in data.iterrows():
        try:
            out = pipe(row[1][column])[0]
            out['target'] = row[1]['target']
            out['type'] = typ
            sent = sent.append(out, ignore_index=True)
        except:
            continue
    return sent

In [7]:
def runDaily():
    targets = getTargets()
    pipe = getPipe()
    tweets = getTweets()
    headlines= getHeadlines()
    sentiments = pd.DataFrame()
    sentiments = sentiments.append(getSentiments(tweets,pipe,'text', 'tweet'))
    sentiments = sentiments.append(getSentiments(headlines, pipe, 'Headline', 'headline'))
    sentiments = sentiments.reset_index(drop=True)
    sentiments.to_csv('data/' +(datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d') + '/sentiments.csv')
    return sentiments

In [8]:
data = runDaily()
data

,target,label,score,type
0,CSCO,neutral,0.999867,tweet
1,CSCO,neutral,0.999851,tweet
2,CSCO,neutral,0.990066,tweet
3,CSCO,neutral,0.999824,tweet
4,CSCO,neutral,0.999573,tweet
...,...,...,...,...
513,SEE,neutral,0.999818,headline
514,SEE,neutral,0.999522,headline
515,SEE,neutral,0.999876,headline
516,SEE,neutral,0.999869,headline
